In [9]:
## imoprts 
import os 
import re
import json
import pandas as pd
from markdown import markdown
from bs4 import BeautifulSoup
from tqdm import tqdm

## input directory 
input_dir = "KT_dataset/"
filenames =  os.listdir(input_dir)
print("Total Notebooks in the dataset = ", len(filenames))

## utility functions
def check_df(df):
    try:
        out = len(df.cell_type.value_counts().index) == 2 and df.cell_type.value_counts().loc["code"] >=1 and df.cell_type.value_counts().loc["markdown"] >=1
    except:
        out = False
    return out

def markdown_to_text(markdown_string):
    """ Converts a markdown string to plaintext """
    html = markdown(markdown_string)
    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code >', ' ', html)
    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(text=True))
    return text

def preprocess_text(text):
    if type(text) == str:
        return text
    else:
        return " ".join(text)
    
## generating data
data = pd.DataFrame(data = [])
count = 0
for i in tqdm(range(len(filenames))):
    filename = filenames[i]
    try:
        ## loading notebook
        notebook = json.load(open(input_dir + filename))
    except:
        ## invlalid files
        continue
        
    df = pd.DataFrame(notebook["cells"], columns = ["source","cell_type"])

    if check_df(df):
        df.dropna(inplace = True)
        df["rank"] = df.index
        df["pct_rank"] = df.index/ len(df)
        df["source"] = df["source"].apply(preprocess_text)
        df["source"] = df["source"].apply(markdown_to_text)
        df["notebook_id"] = i
        data = pd.concat([data, df])
        count += 1

print("Total notebooks used for generating dataset = ", count)
data["rank"] = data["rank"].astype("int")
data["notebook_id"] = data["notebook_id"].astype("int")      
data.to_csv("custom_gen_data.csv", index = False)

Total Notebooks in the dataset =  248761


  7%|█████                                                                    | 17458/248761 [31:42<5:40:10, 11.33it/s]D:\Anaconda\envs\gpu\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
 20%|█████████████▋                                                        | 48525/248761 [1:38:39<21:00:39,  2.65it/s]D:\Anaconda\envs\gpu\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning.

Total notebooks used for generating dataset =  168787
